In [1]:
!pip install surya-ocr

In [2]:
!pip install streamlit
!surya_gui




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.124.148.199:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/bin/surya_gui", line 8, in <module>
    sys.exit(run_app())
  File "/usr/local/lib/python3.10/dist-packages/run_ocr_app.py", line 10, in run_app
    subprocess.run(cmd, env={**os.environ, "IN_STREAMLIT": "true"})
  File "/usr/lib/python3.10/subprocess.py", line 505, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/usr/lib/python3.10/subprocess.py", line 1146, in communicate
    self.wait()
  File "/usr/lib/python3.10/subprocess.py", line 1209, in wait
    return self._wait(timeout=timeout)
  File "/usr/lib/python3.10/subprocess.py", line 1959, in _wait
    (pid, sts) = self._try_wait(0)
  File "/usr/lib/python3.10/subprocess.py", line 1917, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInt

In [20]:
from PIL import Image
from surya.ocr import run_ocr
from surya.model.detection.model import load_model as load_det_model, load_processor as load_det_processor
from surya.model.recognition.model import load_model as load_rec_model
from surya.model.recognition.processor import load_processor as load_rec_processor

image = Image.open("/content/document.png")
langs = ["ne","en"] 
det_processor, det_model = load_det_processor(), load_det_model()
rec_model, rec_processor = load_rec_model(), load_rec_processor()

predictions = run_ocr([image], [langs], det_model, det_processor, rec_model, rec_processor)

Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16


Recognizing Text: 100%|██████████| 1/1 [00:02<00:00,  2.02s/it]


In [21]:
#assuming predictions is a list of results for each image
ocr_results = []

for pred in predictions[0]:  #Process the predictions for the first image
    box, text = pred  #Each prediction contains a bounding box and recognized text
   
    if isinstance(text, list):
        # Convert TextLine objects to strings before joining
        text = " ".join([str(t) for t in text])
    if text.strip():  # Ignore empty text predictions
        ocr_results.append(f"Text: {text}\nPosition: {box}\n")

#save the results to a text file
file_path = "/content/ocr_results (4).txt"
with open(file_path, "w", encoding="utf-8") as file:
    for result in ocr_results:
        file.write(result + "\n")

print(f"OCR results saved to '{file_path}'")


OCR results saved to '/content/ocr_results (4).txt'


In [25]:
!pip install tabled


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 16.9 MB/s eta 0:00:00


In [5]:
from google.colab import files

# Replace 'ocr_results.csv' with the name of your file
files.download("ocr_results.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
list(predictions[0])[0][1]

[TextLine(polygon=[[785.0, 62.0], [898.0, 64.0], [897.0, 82.0], [784.0, 80.0]], confidence=0.98095703125, text='केन्द्रीय कार्यालय', bbox=[785.0, 62.0, 898.0, 82.0]),
 TextLine(polygon=[[764.0, 90.0], [919.0, 90.0], [919.0, 108.0], [764.0, 108.0]], confidence=0.94580078125, text='बालवाटार, काठमाडौँ ।', bbox=[764.0, 90.0, 919.0, 108.0]),
 TextLine(polygon=[[745.0, 122.0], [935.0, 122.0], [935.0, 140.0], [745.0, 140.0]], confidence=0.91943359375, text='फोन नं.: ४४९९८०४/५', bbox=[745.0, 122.0, 935.0, 140.0]),
 TextLine(polygon=[[759.0, 147.0], [921.0, 151.0], [921.0, 168.0], [758.0, 164.0]], confidence=0.994140625, text='Web Site: www.nrb.org.np', bbox=[759.0, 147.0, 921.0, 168.0]),
 TextLine(polygon=[[787.0, 179.0], [892.0, 181.0], [891.0, 198.0], [787.0, 196.0]], confidence=0.95849609375, text='पोष्ट बक्स:७३', bbox=[787.0, 179.0, 892.0, 198.0]),
 TextLine(polygon=[[98.0, 209.0], [526.0, 224.0], [526.0, 251.0], [97.0, 236.0]], confidence=0.9912109375, text='बैंक तथा वित्तीय संस्था नियमन 

In [23]:
#define a class for bounding boxes
class BBox_1:
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max

#define a class for text lines
class TextLine_1:
    def __init__(self, bbox, text):
        self.bbox = bbox
        self.text = text


In [24]:
def my_predictions(data):
    my_data = []
    for pred in data:
        bbox = BBox_1(pred.bbox[0],pred.bbox[1],pred.bbox[2],pred.bbox[3])
        my_data.append(TextLine_1(bbox,pred.text))
    return my_data

In [25]:
def format_text(predictions):
    predictions = my_predictions(list(predictions[0])[0][1])
    #access y_min attribute of MyBBox using dot notation
    data = sorted(predictions, key=lambda x: (x.bbox.y_min))
    prev_y_temp = 0
    for pred in data:
        #access y_min and other attributes using dot notation
        if pred.bbox.y_min - prev_y_temp < 30:
            pred.bbox.y_min = prev_y_temp
        else:
            prev_y_temp = pred.bbox.y_min
    #access y_min and x_min attributes for sorting
    data = sorted(data, key=lambda x: (x.bbox.y_min, x.bbox.x_min))
    formatted_text = ''
    #access x_min attribute for minimum calculation
    min_x = min(data, key=lambda x: x.bbox.x_min).bbox.x_min
    min_y = data[0].bbox.y_min
    prev_x_max = 0
    prev_y_max = 0
    prev_y = 0
    for pred in data:
        #access bbox attributes using dot notation
        current_x = pred.bbox.x_min - min_x
        current_y = pred.bbox.y_min - min_y
        current_y_max = pred.bbox.y_max - min_y
        line_breaks = '\n' * int(((current_y - prev_y_max) / 60))
        if line_breaks:
            prev_x_max = 0
        #blanks = ' ' * int(((current_x - prev_x_max) / 8))
        formatted_text += line_breaks
        #formatted_text += blanks
        formatted_text += ' ' * int(current_x / 15)
        formatted_text += pred.text + '\n'
        prev_x_max = pred.bbox.x_max
        prev_y = current_y
        prev_y_max = current_y_max
    return formatted_text

In [26]:
final1 = format_text(predictions)
print(final1)

                                              बालवाटार, काठमाडौँ ।
                                               केन्द्रीय कार्यालय
                                             फोन नं.: ४४९९८०४/५
                                              Web Site: www.nrb.org.np
                                                पोष्ट बक्स:७३
  बैंक तथा वित्तीय संस्था नियमन विभाग
 पत्र  संख्या:  बै.वि.नि.वि. / नीति / परिपत्र / ०४ / ०८० / ५१
                                               मिति: २०८०/०६/३०
 इजाजतपत्रप्रप्राप्त “क”, “ख” र “ग” वर्गका बैंक तथा वित्तीय संस्थाहरु,
 महाशय.
   यस वैंकवाट “क”, “ख” र “ग” वर्गका इजाजतपत्रप्राप्त बैंक तथा वित्तीय संस्थाहरुलाई जारी
 तथा संशोधनहरु एकीकृत गरी तयार पारिएको यसै साथ संलग्न एकीक्त निर्देशन, २०५० जारी गरिएको
 गरिएको एकीकृत निर्देशन, २०७९ तथा सो पश्चीत २०८० असोज मसान्तसम्म जारी गरिएका परिपत्र
 हुँदा सोहीबमोजिम गर्नु/गराउनु हुन नेपाल राष्ट्र बैंक ऐन, २०५८ को दफा ७९ ले दिएको अधिकार
 प्रयोग गरी यो निर्देशन जारी गरिएको छ ।


                                   

In [14]:
from google.colab import files

#save the content of 'this' as a text file
with open('output.txt', 'w') as file:
    file.write(this)

#downloading the file
files.download('output.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>